In [2]:
import json

languages = ['go', 'java', 'javascript', 'php', 'python', 'ruby']

for language in languages:
    # 指定JSONL文件的路径
    jsonl_file_path = f'{language}/train.jsonl'

    # 创建一个空字典来存储每个仓库的数量
    repo_counts = {}

    # 打开JSONL文件以读取内容
    with open(jsonl_file_path, 'r') as file:
        # 逐行读取文件内容
        for line in file:
            # 解析JSON对象
            json_object = json.loads(line)
            
            # 获取"repo"键的值
            if "repo" in json_object:
                repo_value = json_object["repo"]
                
                # 更新仓库数量
                if repo_value in repo_counts:
                    repo_counts[repo_value] += 1
                else:
                    repo_counts[repo_value] = 1

    # 按仓库名排序字典
    sorted_repo_counts = dict(sorted(repo_counts.items(), key=lambda x: x[0]))

    # 打印每个仓库及其对应的数量
    for repo, count in sorted_repo_counts.items():
        print(f"仓库: {repo}, 数量: {count}")

仓库: 0x19/goesl, 数量: 19
仓库: 0xAX/notificator, 数量: 1
仓库: 0xfaded/eval, 数量: 30
仓库: 0xor1/joak, 数量: 19
仓库: 0xrawsec/golang-evtx, 数量: 32
仓库: 0xrawsec/golang-utils, 数量: 79
仓库: 10gen/openssl, 数量: 3
仓库: 1and1/oneandone-cloudserver-sdk-go, 数量: 16
仓库: 256dpi/gomqtt, 数量: 144
仓库: 30x/authsdk, 数量: 6
仓库: 360EntSecGroup-Skylar/excelize, 数量: 121
仓库: 42wim/matterbridge, 数量: 58
仓库: 42wim/matterbridge-plus, 数量: 2
仓库: 4ydx/gltext, 数量: 15
仓库: 99designs/basicauth-go, 数量: 1
仓库: 99designs/httpsignatures-go, 数量: 5
仓库: 99designs/keyring, 数量: 7
仓库: APTrust/bagins, 数量: 11
仓库: AaronO/go-git-http, 数量: 14
仓库: Ableton/go-travis, 数量: 6
仓库: AcalephStorage/consul-alerts, 数量: 10
仓库: AkihiroSuda/go-netfilter-queue, 数量: 4
仓库: AlecAivazis/survey, 数量: 21
仓库: AlekSi/reflector, 数量: 2
仓库: AlekSi/zabbix, 数量: 12
仓库: AlexanderThaller/logger, 数量: 7
仓库: AliyunContainerService/docker-machine-driver-aliyunecs, 数量: 2
仓库: AndrewBurian/powermux, 数量: 51
仓库: ArchCI/archci, 数量: 41
仓库: AreaHQ/go-fixtures, 数量: 6
仓库: ArthurHlt/go-eureka-client

In [1]:
import json
import os

# 定义要处理的语言列表
languages = ['go', 'java', 'javascript', 'php', 'python', 'ruby']

for language in languages:
    # 指定JSONL文件的路径
    jsonl_file_path = f'{language}/train.jsonl'
    output_jsonl_file = f'{language}/train_sorted.jsonl'

    # 打开输入的JSONL文件以读取内容
    with open(jsonl_file_path, 'r') as input_file:
        # 创建一个空列表来存储每行内容
        lines = input_file.readlines()

    # 解析每行内容，并按仓库名排序
    sorted_lines = sorted(lines, key=lambda line: json.loads(line).get("repo", ""))

    # 打开输出的JSONL文件以写入内容
    with open(output_jsonl_file, 'w') as output_file:
        # 遍历排序后的行并写入输出文件
        for line in sorted_lines:
            output_file.write(line)


In [1]:
import json
import os

# 定义要处理的语言列表
languages = ['go', 'java', 'javascript', 'php', 'python', 'ruby']

for language in languages:
    # 指定JSONL文件的路径
    jsonl_file_path = f'{language}/train_sorted.jsonl'
    output_folder = f'{language}/split_jsonl_files'

    num = 0

    # 创建输出文件夹（如果不存在）
    os.makedirs(output_folder, exist_ok=True)

    # 创建一个字典来存储每个用户的 JSON 对象列表
    user_objects = {}

    # 打开输入的JSONL文件以读取内容
    with open(jsonl_file_path, 'r') as input_file:
        for line in input_file:
            num = num + 1
            # 解析JSON对象
            json_object = json.loads(line)

            # 获取"repo"项的值，格式为'用户名/仓库名'
            repo_value = json_object.get("repo", "")

            # 提取用户名
            username = repo_value.split('/')[0]

            # 如果当前用户不在字典中，则创建一个新的用户列表
            if username not in user_objects:
                user_objects[username] = []

            # 将JSON对象添加到当前用户的列表中
            user_objects[username].append(json_object)

    # 计算每个子数据集应包含的 JSON 数量
    jsons_per_subset = num // 10

    # 初始化子数据集和文件编号
    current_subset = []
    file_number = 0

    # 输出每个数据集中的 JSON 条数信息
    print(f"Language: {language}")
    print(f"JSONs per Subset: {jsons_per_subset}")

    # 遍历用户及其JSON对象列表，将它们分配到子数据集中
    for username, user_jsons in user_objects.items():
        current_subset.extend(user_jsons)

        # 如果当前子数据集达到或超过应包含的 JSON 数量
        if len(current_subset) >= jsons_per_subset:
            # 创建输出文件
            output_file_path = os.path.join(output_folder, f'subset_{file_number}.jsonl')
            with open(output_file_path, 'w') as output_file:
                for json_object in current_subset:
                    # 将JSON对象写入输出文件
                    json.dump(json_object, output_file)
                    output_file.write('\n')

            # 输出当前子数据集中的 JSON 条数信息
            print(f"Subset {file_number}: {len(current_subset)} JSONs")

            # 清空当前子数据集，增加文件编号
            current_subset = []
            file_number += 1

    # 将剩余的JSON对象写入最后一个输出文件
    if current_subset:
        output_file_path = os.path.join(output_folder, f'subset_{file_number}.jsonl')
        with open(output_file_path, 'w') as output_file:
            for json_object in current_subset:
                json.dump(json_object, output_file)
                output_file.write('\n')
        
        # 输出最后一个子数据集中的 JSON 条数信息
        print(f"Subset {file_number}: {len(current_subset)} JSONs")


Language: go
JSONs per Subset: 16728
Subset 0: 19064 JSONs
Subset 1: 16731 JSONs
Subset 2: 16729 JSONs
Subset 3: 18476 JSONs
Subset 4: 19408 JSONs
Subset 5: 16948 JSONs
Subset 6: 16772 JSONs
Subset 7: 16729 JSONs
Subset 8: 16841 JSONs
Subset 9: 9590 JSONs
Language: java
JSONs per Subset: 16492
Subset 0: 17302 JSONs
Subset 1: 17148 JSONs
Subset 2: 17511 JSONs
Subset 3: 16559 JSONs
Subset 4: 20113 JSONs
Subset 5: 16533 JSONs
Subset 6: 16591 JSONs
Subset 7: 16515 JSONs
Subset 8: 16577 JSONs
Subset 9: 10074 JSONs
Language: javascript
JSONs per Subset: 5802
Subset 0: 5866 JSONs
Subset 1: 6245 JSONs
Subset 2: 5817 JSONs
Subset 3: 6229 JSONs
Subset 4: 5817 JSONs
Subset 5: 5805 JSONs
Subset 6: 5833 JSONs
Subset 7: 5802 JSONs
Subset 8: 5824 JSONs
Subset 9: 4787 JSONs
Language: php
JSONs per Subset: 24124
Subset 0: 24189 JSONs
Subset 1: 24124 JSONs
Subset 2: 24150 JSONs
Subset 3: 24126 JSONs
Subset 4: 24606 JSONs
Subset 5: 25462 JSONs
Subset 6: 24201 JSONs
Subset 7: 24189 JSONs
Subset 8: 24262 J